In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Dataset
import pandas as pd
import time
import os
from torchvision import transforms
from PIL import Image

In [2]:
# paths to dataset and csv
CSV_PATH = 'top50.csv'
IMAGE_FOLDER_PATH = 'top50images'

In [3]:
class CityImageDataset(Dataset):
    """
    Custom pytorch dataset for our top 50 cities dataset
    """
    
    def __init__(self, csv_path, image_folder, transform=None):
        """
        Args:
            csv_path: path to the csv file with labels.
            image_folder: directory of images
            transform: transform to be applied on images.
        """
        self.img_labels = pd.read_csv(csv_path)
        self.image_folder = image_folder
        self.transform = transform
        
        # encode cities to labels
        self.unique_cities = self.img_labels['city'].unique()
        self.city_to_idx = {city: i for i, city in enumerate(self.unique_cities)}
        self.idx_to_city = {i: city for i, city in enumerate(self.unique_cities)}
        
        print(f"Mapped {len(self.unique_cities)} unique classes (cities)") # should be 50
        # print(self.city_to_idx)

    def __len__(self):
        """
        Returns length of the dataset (number of images)
        """
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Get a data sample for a given index.

        Args:
            idx: index of item/image

        Returns:
            tuple: (image, label) where image is the transformed image tensor
                   and label is the encoded label of the city.
        """
        # get image file name and its city from dataframe
        row = self.img_labels.iloc[idx]
        image_filename = row['filename']
        city_label_str = row['city']
        
        # construct path to image
        image_path = os.path.join(self.image_folder, image_filename)
        
        # load image
        try:
            image = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Error, could not find image at path: {image_path}")
            return None, -1 

        # get encoded label of city
        label = self.city_to_idx[city_label_str]

        # apply necessary transformations
        if self.transform:
            image = self.transform(image)
            
        return image, label
    

In [4]:
# transform data for resnet
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),      # resize to 224 x 224
    transforms.ToTensor(),
    transforms.Normalize(               # normalize to imagenet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# create dataset
print("Creating dataset...")
full_dataset = CityImageDataset(
    csv_path=CSV_PATH,
    image_folder=IMAGE_FOLDER_PATH,
    transform=data_transforms
)

# split data into 80/20 for train/val
dataset_size = len(full_dataset)
train_size = int(dataset_size * 0.8)
val_size = dataset_size - train_size

Creating dataset...
Mapped 50 unique classes (cities)


In [5]:
# seed and split
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

print(f"Total samples: {dataset_size}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Total samples: 8023
Training samples: 6418
Validation samples: 1605


In [6]:
# weight samples to try to have balanced training batches
# get labels for training set samples
train_labels = [full_dataset.img_labels.iloc[i]['city'] for i in train_dataset.indices]

# count occurrences of each class
class_counts = pd.Series(train_labels).value_counts().sort_index()
print(f"Class counts in training set: {class_counts}")

# calc weight for each class
class_weights = 1.0 / torch.tensor(class_counts.values, dtype=torch.float)

# create weight for each training sample
sample_weights = torch.tensor([class_weights[full_dataset.city_to_idx[label]] for label in train_labels])

# create weighted sampler
train_sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

Class counts in training set: Albuquerque         132
Atlanta             137
Austin              142
Bakersfield         147
Baltimorem           79
Boston               46
Charlotte           146
Chicago             128
Colorado Springs    127
Columbus            154
Dallas              158
Denveri             144
Detroit             136
El Paso              75
Fort Worth          111
Fresno              123
Houston             142
Indianapolisg       140
Jacksonvillef       145
Kansas City         141
Las Vegas           125
Long Beach          124
Los Angeles         147
Louisvillel         120
Memphis             147
Mesa                 91
Miami               151
Milwaukee           152
Minneapolis         148
Nashvillej          134
New Yorkd           152
Oakland             119
Oklahoma City       142
Omaha               138
Philadelphiae       161
Phoenix             141
Portland            154
Raleigh             146
Sacramento          142
San Antonio         163
San Diego 

In [7]:
# create data loaders
BATCH_SIZE = 32

# training set loader
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,  # let weighted sampler do sampling
    num_workers=0,          # change to higher number on colab since mac is dumb
    shuffle=False           # need to set shuffle to false when using a sampler
)

# validation set loader
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0           # change to higher number on colab since mac is dumb
)

print(f"Batch size: {BATCH_SIZE}")
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")

Batch size: 32
Number of training batches: 201
Number of validation batches: 51


In [8]:
# get device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


In [ ]:
def train_and_validate(model, loss_func, optimizer, train_loader, val_loader, device, num_epochs=5):
    """
    Args:
        model: model to train
        loss_func: loss function
        optimizer: optimizer
        train_loader: DataLoader for the training set
        val_loader: DataLoader for the validation set
        device: the torch.device to run on
        num_epochs: number of epochs
    """
    print()
    print("--- Starting Training ---")
    for epoch in range(num_epochs):

        # keep track of how much time each epoch takes
        start_time = time.time()
        
        # TRAINING
        model.train()  # set to training mode
        running_loss = 0.0  # keep track of loss

        # iterate through batches
        for images, labels in train_loader:

            # move to device
            images, labels = images.to(device), labels.to(device)
            
            # zero out gradients from previous batch
            optimizer.zero_grad()

            # forward pass
            outputs = model(images)

            # apply loss function
            loss = loss_func(outputs, labels)

            # backprop
            loss.backward()

            # update weights
            optimizer.step()
            
            # keep track of loss
            running_loss += loss.item() * images.size(0)
        
        # keep track of total epoch loss
        epoch_train_loss = running_loss / len(train_loader.sampler)

        # VALIDATION
        model.eval()  # set to eval mode
        val_loss = 0.0
        correct_predictions = 0

        # no gradients
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                
                # apply forward pass and evaluate output
                outputs = model(images)
                loss = loss_func(outputs, labels)
                val_loss += loss.item() * images.size(0)
                
                # get prediction
                _, preds = torch.max(outputs, 1)
                correct_predictions += torch.sum(preds == labels.data)
        
        # keep track of total epoch val loss
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = correct_predictions.float() / len(val_loader.dataset)
        
        # time it took for epoch to train
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch + 1}/{num_epochs} | Time: {epoch_time:.2f}s")
        print(f"\t Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")

    print()
    print("--- Training Complete ---")


In [ ]:
# load resnet model
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# model = models.resnet50(weights='DEFAULT')

# freeze params/layers of model
print("Freezing parameters of pre-trained layers...")
for param in resnet50.parameters():
    param.requires_grad = False

# replace final layer with our classification layer
num_features = resnet50.fc.in_features
num_classes = len(full_dataset.unique_cities)
resnet50.fc = nn.Linear(num_features, num_classes)
print(f"Replaced final layer, new # of output features: {num_classes}")

Freezing parameters of pre-trained layers...
Replaced final layer, new # of output features: 50


In [11]:
# move model to device
resnet50 = resnet50.to(device)

In [12]:
# hyperparams
NUM_EPOCHS = 5
LEARNING_RATE = 0.001

In [13]:
# set up loss func and optimizer

# loss func
loss_func = nn.CrossEntropyLoss()

# will only update weights for final layer since we froze previous ones
optimizer = optim.Adam(resnet50.fc.parameters(), lr=LEARNING_RATE)

In [14]:
# training/validation loop
train_and_validate(resnet50, loss_func, optimizer, train_loader, val_loader, device, NUM_EPOCHS)


--- Starting Training ---
Epoch 1 / 5 | Time: 127.35s
	 Train Loss: 3.3283 | Val Loss: 3.3198 | Val Acc: 0.1614
Epoch 2 / 5 | Time: 127.18s
	 Train Loss: 2.7179 | Val Loss: 3.1107 | Val Acc: 0.2037
Epoch 3 / 5 | Time: 126.48s
	 Train Loss: 2.4101 | Val Loss: 3.0205 | Val Acc: 0.2336
Epoch 4 / 5 | Time: 124.62s
	 Train Loss: 2.1686 | Val Loss: 2.9591 | Val Acc: 0.2336
Epoch 5 / 5 | Time: 120.23s
	 Train Loss: 1.9716 | Val Loss: 2.9407 | Val Acc: 0.2417

--- Training Complete ---


In [15]:
# Freezing parameters of pre-trained layers...
# Replaced final layer. Output features: 50
# Epoch 1/5 | Time: 125.85s
# 	Train Loss: 3.3243 | Val Loss: 3.3010 | Val Acc: 0.1657
# Epoch 2/5 | Time: 120.36s
# 	Train Loss: 2.7309 | Val Loss: 3.0988 | Val Acc: 0.2206
# Epoch 3/5 | Time: 120.52s
# 	Train Loss: 2.4108 | Val Loss: 3.0306 | Val Acc: 0.2206
# Epoch 4/5 | Time: 120.46s
# 	Train Loss: 2.1759 | Val Loss: 2.9664 | Val Acc: 0.2374
# Epoch 5/5 | Time: 121.21s
# 	Train Loss: 1.9838 | Val Loss: 2.9176 | Val Acc: 0.2480